# Features Extracting and basic data formating

In [1]:
import spotipy
import pandas as pd
import plotly.express as px
import seaborn as sns
import sys
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Adding client_id and client_secret for the list

In [3]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id='c7832f4cfebb4553b27447fe32b29c1d', client_secret='d61c1c95e44446bdab707511f8d306ae')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
#Accessing my own play list https://open.spotify.com/playlist/73EgGtgIWyZNRmFyIt11iQ?si=477933e6f12b431c

In [5]:
playlist_link = "https://open.spotify.com/playlist/2QECvpSDU035PFS5uJ8x3l?si=5730077d7aca4db3"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [6]:
# Here to understand spotify data set with data we have

In [7]:
# Function to extract MetaData from a playlist thats longer than 100 songs
def get_playlist_tracks_more_than_100_songs(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        print(i) # Counter
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
                           'title':[playlist_tracks_titles],
                           'all_artists':[playlist_tracks_artists],
                           'popularity':[playlist_tracks_popularity],
                           'release_date':[playlist_tracks_first_release_date],
                           'danceability':[features[0]['danceability']],
                           'energy':[features[0]['energy']],
                           'key':[features[0]['key']],
                           'loudness':[features[0]['loudness']],
                           'mode':[features[0]['mode']],
                           'acousticness':[features[0]['acousticness']],
                           'instrumentalness':[features[0]['instrumentalness']],
                           'liveness':[features[0]['liveness']],
                           'valence':[features[0]['valence']],
                           'tempo':[features[0]['tempo']],
                           'duration_ms':[features[0]['duration_ms']],
                           'time_signature':[features[0]['time_signature']]
                           }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue

    return features_df

In [8]:
get_playlist_tracks_more_than_100_songs('c7832f4cfebb4553b27447fe32b29c1d','2QECvpSDU035PFS5uJ8x3l')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5MCLya9AzV9spNPxxFk7JD,سلم يا سلام,ايمان الشريف,26,2021-12-09,0.689,0.698,8,-5.818,1,0.0656,0.000005,0.0544,0.909,159.888,220823,4
1,43tNfS6NwbxWJN7z9GD4Rb,وليد من الشكرية,ايمان الشريف,37,2020-03-21,0.741,0.665,10,-3.923,1,0.3110,0.000000,0.0860,0.941,140.010,175125,4
2,1Xv6wXtCnR1sAvC8VbbJcY,نادو لي عشاي,ايمان الشريف,28,2020-09-21,0.668,0.803,0,-2.773,1,0.1550,0.000000,0.0571,0.884,155.004,166113,4
3,42xuLrtlrpYkJQUqchohpS,واثقه في زولي,ايمان الشريف,24,2021-07-19,0.726,0.720,11,-6.761,0,0.1200,0.000002,0.0296,0.967,154.610,206968,3
4,4JoJdgE7epKuvs7WsJDY6I,مالو,ايمان الشريف,24,2020-06-10,0.709,0.688,9,-5.910,1,0.2490,0.000098,0.1930,0.694,135.026,201535,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,4sdLw39RTAekV8xIn03WgI,سقف الصبر,محمد النصري,4,2017-08-30,0.601,0.498,9,-12.274,0,0.6310,0.001330,0.1110,0.928,77.865,397636,4
521,6C7YAXTBvuSJE5M2n9nN05,بلدي,محمد النصري,4,2018-04-22,0.552,0.666,9,-4.561,1,0.8160,0.041800,0.0972,0.961,70.261,785231,4
522,02U09D4oVK6K314Y0H07Td,ست الدار,محمد النصري,5,2019-05-10,0.524,0.609,11,-4.292,1,0.8980,0.000012,0.0740,0.839,80.342,992967,4
523,318yxr3hO4q1D9MAFF0g0k,حُرقة و شجن,محمد النصري,4,2018-07-23,0.620,0.344,9,-13.940,1,0.8570,0.004950,0.1060,0.930,144.115,399517,4
